# Refrigeration cycle Analysis - OOP

* expression only


* Object-oriented programming

## 1 An ideal vapor-compression refrigeration cycle

* https://www.cpp.edu/~tknguyen/che302/home.htm

  * https://www.cpp.edu/~tknguyen/che302/Notes/chap7-2.pdf

Refrigerant 134a is the working fluid in an ideal vapor-compression refrigeration cycle that
communicates thermally with a cold region at 0°C and a warm region at 26°C. 

Saturated vapor enters the compressor at 0°C and saturated liquid leaves the condenser at 26°C.

Themass flow rate of the refrigerant is 0.08 kg/s.

**Determine** 

* (a) the compressor power, in kW,

* (b) the refrigeration capacity, in tons, 

* (c) the coefficient of performance 

* (d) the coefficient of performance of a Carnot refrigeration cycle operating between warm and cold regions at 26 and 0°C

![](./img/vpc.jpg)



(a) The compressor work is give by

$\dot{W}_{c}=\dot{m}*(h_2-h_1)$

(b) The refrigeration capacity, in tons, is

$\dot{Q}_{L}=\dot{m}*(h_1-h_4)*(60s/min)\frac{1ton}{211 kJ/min}$


(c) The coefficient of performance is

$\beta=\frac{\dot{Q}_{L}}{\dot{W}_{c}}=\frac{h_1-h_4}{h_2-h_1}$

## 2 The Object-oriented Programming of Refrigeration cycle

 Modeling and Simulation of the Rankine Cycle with [Computational Thinking](./Unit3-1-CLASSES_AND_OBJECT-ORIENTED_PROGRAMMING.ipynb) to the `generic` solutions


### 2.1 The Refrigeration Cycle 

Apply **abstraction** and **decomposition** to code rankine cycle 8.1&8.2 simulator

```python
   
"""
    ----Node 1--- Compressor---Node 2------------
    |                                   |
 Evaporator                                  Condenser
    |                                      |
    ----Node 4---Expansion Valve ----Node 3-------- 
"""    
```

**Decomposition** : Decompose The ideal refrigeration cycle into parts ： `nodes and devices` 


**Abstraction**  : Define the classes of nodes and devices : `data and methods`


* **1** - Node 

* **2** - Compressor， Condenser，Expansion Valve ， Evaporator


**Algorithms** : obtain the solutions



### 2.2 The classes of nodes and devices 



####  Package :components

 `./refrigeration/components/



In [1]:
%%file ./refrigeration/components/__init__.py
"""
    Components Package  
"""

from .node import Node

from .compressor import  Compressor
from .condenser import  Condenser
from .expansionvalve  import  ExpansionValve
from .evaporator  import  Evaporator



# ------------------------------------------------------------------------------
# compdict(jump table)
#  1: key:value-> Type String: class  name
#  2    add the new key:value to the dict after you add the new device class/type
# --------------------------------------------------------------------------------

compdict = {
    "COMPRESSOR": Compressor,
    "CONDENSER": Condenser,
    "EXPANSIONVALVE":ExpansionValve,
    "EVAPORATOR":Evaporator
} 



Overwriting ./refrigeration/components/__init__.py


### 2.3 Node Class

* **Properties:** name,id, p,t,h,s,x,mdot

* **Methods:** (p,s),(t,x),setstate, `__str__`

In [2]:
%%file ./refrigeration/components/node.py

import CoolProp.CoolProp as cp

class Node:

    title = ('{:^6} \t {:^30} \t {:<3}\t {:>10}\t {:>10}\t {:>10} \t {:>5}\t {:>15}'.format
             ("NodeID", "Name", "P(MPa)", "T(°C)", "H(kJ/kg)", "S(kJ/kg.K)",  "X", "MDOT(kg/s)"))

    def __init__(self, dictnode):
        """ create the node object"""

        self.name = dictnode['name']
        self.id = dictnode['id']

        try:
            self.t = float(dictnode['t'])
        except:
            self.t = None
        try:
            self.x = float(dictnode['x'])
        except:
            self.x = None

        try:
            self.mdot = float(dictnode['mdot'])
        except:
            self.mdot = None

        self.p = None
        self.h = None
        self.s = None
        self.stateok = False

        if self.t is not None and self.x is not None:
            self.tx()

    def tx(self):
        self.p = cp.PropsSI('P', 'T', 273.15+self.t,
                            'Q', self.x, 'R134a')/1.0e6
        self.h = cp.PropsSI('H', 'T', 273.15+self.t, 'Q', self.x, 'R134a')/1000
        self.s = cp.PropsSI('S', 'T', 273.15+self.t, 'Q', self.x, 'R134a')/1000
        self.stateok = True

    def ps(self):
        self.h = cp.PropsSI('H', 'P', self.p*1.0e6, 'S',self.s*1000, 'R134a')/1000
        self.t = cp.PropsSI('T', 'P', self.p*1.0e6, 'S',self.s*1000, 'R134a')-273.15
        self.stateok = True


    def setstate(self):
        if self.t is not None and self.x is not None:
            self.tx()
        if self.p is not None and self.s is not None:
            self.ps()

    def __str__(self):
        result = ('{:^6} \t {:<30}'.format(self.id, self.name))
            
        OutStrs=[ {"fstr":'\t {:<6.3}',"sstr":'\t {:<6}','prop':self.p},
                  {"fstr":'\t {:>10.2f}',"sstr":'\t {:>10}','prop':self.t},
                  {"fstr":'\t {:>10.2f}',"sstr":'\t {:>10}','prop':self.h},
                  {"fstr":'\t {:>7.2f}',"sstr":'\t {:>7}','prop':self.s},
                  {"fstr":'\t {:>10.2f}',"sstr":'\t {:>10}','prop':self.x},
                  {"fstr":'\t {:>12.2f}',"sstr":'\t {:>12}','prop':self.mdot}
                ]
                     
        for item in OutStrs: 
            try:
                result +=item["fstr"].format(item["prop"])
            except:
                result += item["sstr"].format("")
            
     
        return result


Overwriting ./refrigeration/components/node.py


### 2.4 Device Classes
 
####  Compressor Class:

```

                        ↑    oNode 
                ┌───┼───┐
                │      │      │
                │      │      │
                │      │      │
    compressor work  └───┼───┘  
                        ↑    iNode  
```  
* **Properties:**  
 
  * iNode，oNode
  
  * compressor work

* **Process:**  
  
  * Isentropic compression of the refrigerant from state inletNode  to state exitNode


In [3]:
%%file ./refrigeration/components/compressor.py


from .node import *

class Compressor:
    """ Isentropic compression of the refrigerant"""
    energy = "CompressionWork"
    devtype = "COMPRESSOR"

    def __init__(self, dictDev, nodes):
        """
        Initializes 
        """
        self.name = dictDev['name']
        self.iNode = nodes[dictDev['iNode']]
        self.oNode = nodes[dictDev['oNode']]
        self.Wc=None
  
    def state(self):
        """
        Isentropic compression of the refrigeran
        """
        self.oNode.s=self.iNode.s
                
    def balance(self):
        """  mass and energy balance    """
        # mass balance
        if self.iNode.mdot is not None:
            self.oNode.mdot = self.iNode.mdot
        elif self.oNode.mdot is not None:
            self.iNode.mdot = self.oNode.mdot
        # energy
        self.Wc = self.iNode.mdot * (self.oNode.h - self.iNode.h)

  
    def __str__(self):
        result = '\n' + self.name
        result += '\n' + Node.title
        result += '\n' + self.iNode.__str__()
        result += '\n' + self.oNode.__str__()
        result += '\nWc(kW): \t{:>.2f}'.format(self.Wc)
        return result


Overwriting ./refrigeration/components/compressor.py


#### Condenser Class

The Condenser 
```
                           
                ┌──────┐
                │            │ 
     oNode   ←┼──────┼← iNode
                │            │
                └──────┘  
                            

```
* **Properties:**  
 
  * iNode，oNode 
  


In [4]:
%%file ./refrigeration/components/condenser.py


from .node import *
import CoolProp.CoolProp as cp

class Condenser:

    energy = "None"
    devtype = "CONDENSER"

    def __init__(self, dictDev, nodes):
        """ Initializes the condenser """
        self.name = dictDev['name']
        self.iNode = nodes[dictDev['iNode']]
        self.oNode = nodes[dictDev['oNode']]

    def state(self):
        self.oNode.p=cp.PropsSI('P', 'T', 273.15+self.oNode.t, 'Q',self.oNode.x, 'R134a')/1.0e6
        self.iNode.p=self.oNode.p
        
    def balance(self):
        """ mass and energy balance of the condenser  """
        if self.iNode.mdot is not None:
            self.oNode.mdot = self.iNode.mdot
        elif self.oNode.fdot is not None:
            self.iNode.mdot = self.oNode.mdot

  
    def __str__(self):
        result = '\n' + self.name
        result += '\n' + Node.title
        result += '\n' + self.iNode.__str__()
        result += '\n' + self.oNode.__str__()
        return result



Overwriting ./refrigeration/components/condenser.py


#### Expansion Valve Class


```    
                ┌───────┐
                │              │
        oNode  ← ┼───────┼← iNode
               │               │
                └───────┘  
```
* **Properties:**  
 
  * iNode，oNode
  


In [5]:
%%file ./refrigeration/components/expansionvalve.py


from .node import *

class ExpansionValve:

    energy = "None"
    devtype = "EXPANSIONVALVE"

    def __init__(self, dictDev, nodes):
        """ Initializes the ExpansionValve """
        self.name = dictDev['name']
        self.iNode = nodes[dictDev['iNode']]
        self.oNode = nodes[dictDev['oNode']]

    def state(self):
        self.oNode.h=self.iNode.h
    
    def balance(self):
        """ mass and energy balance  """
        if self.iNode.mdot is not None:
            self.oNode.mdot = self.iNode.mdot
        elif self.oNode.fdot is not None:
            self.iNode.mdot = self.oNode.mdot

 
    def __str__(self):
        result = '\n' + self.name
        result += '\n' + Node.title
        result += '\n' + self.iNode.__str__()
        result += '\n' + self.oNode.__str__()
        return result


Overwriting ./refrigeration/components/expansionvalve.py


#### Evaporator Class

The Evaporator 
```
                          
                ┌──────┐
                │            │ 
   oNode   ←┼──────┼← iNode
                │            │
                └──────┘  
                          

```
* **Properties:**  
 
  * iNode，oNode 
  
  * Qlow


In [6]:
%%file ./refrigeration/components/evaporator.py


from .node import *

class  Evaporator:

    energy = "RefrigerationCapacity"
    devtype = "EVAPORATOR"

    def __init__(self, dictDev, nodes):
        """ Initializes the Evaporator """
        self.name = dictDev['name']
        self.iNode = nodes[dictDev['iNode']]
        self.oNode = nodes[dictDev['oNode']]

    def state(self):
        self.iNode.t=self.oNode.t

    def balance(self):
        """ mass and energy balance  """
        if self.iNode.mdot is not None:
            self.oNode.mdot = self.iNode.mdot
        elif self.oNode.fdot is not None:
            self.iNode.mdot = self.oNode.mdot
        self.Qlow= self.iNode.mdot * (self.oNode.h - self.iNode.h)
 
    def __str__(self):
        result = '\n' + self.name
        result += '\n' + Node.title
        result += '\n' + self.iNode.__str__()
        result += '\n' + self.oNode.__str__()
        result += '\nQlow(kW): \t{:>.2f}'.format(self.Qlow)    
       
        return result



Writing ./refrigeration/components/evaporator.py


### 2.5 Analysis the Refrigeration  Cycle 
  
* 1 init nodes

* 2 connect device

* 3 simulate devices

* 4 cycle

#### cycleModel

In [8]:
%%file ./refrigeration/cyclemodel/__init__.py
from . import ivpr1

cycles = [ivpr1]


Overwriting ./refrigeration/cyclemodel/__init__.py


In [9]:
%%file ./refrigeration/cyclemodel/ivpr1.py
cycle = {}
cycle["name"] = "Ideal_VPR"
cycle["nodes"] = [{
    "name": "Evaporator to Compressor",
            "id": 1,
            "t": 0.0,
            "x": 1,
            "mdot": 0.08
 },
    {
    "name": "Compressor to  Condenser",
            "id": 2,
            "t": None,
            "x": None,
            "mdot": None
},
    {
    "name": "Condenser to Expansion Valve ",
            "id": 3,
            "t": 26.0,
            "x": 0,
            "mfdot": None
},
    {
    "name": "Expansion Valve to Evaporator",
            "id": 4,
            "t": None,
            "x": None,
            "mdot": None
}
]

cycle["comps"] = [
    {
        "name": "Compressor",
        "devtype": "COMPRESSOR",
        "iNode": 1,
        "oNode": 2
    },
    {
        "name": "Condenser",
        "devtype": "CONDENSER",
        "iNode": 2,
        "oNode": 3
    },
    {
        "name": "Expansion Valve ",
        "devtype": "EXPANSIONVALVE",
        "iNode": 3,
        "oNode": 4
    },
    {
        "name": "Evaporator",
        "devtype": "EVAPORATOR",
        "iNode": 4,
        "oNode": 1
    }
]


Writing ./refrigeration/cyclemodel/ivpr1.py


#### vprcycle



In [10]:
%%file ./refrigeration/vprcycle/__init__.py

"""
 General Object-oriented Abstraction of vpr Cycle 
    
    cycle Package 

"""

import sys
sys.path.append('../')


Overwriting ./refrigeration/vprcycle/__init__.py


In [11]:
%%file ./refrigeration/vprcycle/cycleobj.py
"""
 General Object-oriented Abstraction of vpr Cycle 

RefrigerationCycle: the Simulator class of VPR Cycle  

dictcycle={"name":namestring,
                     "nodes":[{node1},{node2},...],
                     "comps":[{component1},{component2},...]
                     }
  
Author:Cheng Maohua  Email: cmh@seu.edu.cn

"""

import time

from components.node import Node
from components import compdict


class RefrigerationCycle:

    def __init__(self, dictcycle):
        """
          dictcycle={"name":namestring,
                     "nodes":[{node1},{node2},...],
                     "comps":[{component1},{component2},...]
                     }
          TO:           
             self.nodes : dict of all node objects
             self.comps : dict of all component objects
        """
        self.name = dictcycle["name"]
        dictnodes = dictcycle["nodes"]
        dictcomps = dictcycle["comps"]

        # 1 convert dict to the object of nodes
        self.NodeNum = len(dictnodes)
        self.nodes = {}
        for curnode in dictnodes:
            self.nodes[curnode["id"]] = Node(curnode)

        # 2 convert dict to the object of Comps
        self.DevNum = len(dictcomps)
        self.comps = {}
        for curdev in dictcomps:
            self.comps[curdev['name']] = compdict[curdev['devtype']](
                curdev, self.nodes)
       
        self.Wc = 0
        self.Qlow = 0
        self.cop = 0.0

       
    def ComponentState(self):
        for key in self.comps:
            self.comps[key].state()
       
        for key in self.nodes:
            if self.nodes[key].stateok==False:
              self.nodes[key].setstate()

    def ComponentBalance(self):
        for curdev in self.comps:
            self.comps[curdev].balance()
    
    def simulator(self):
        self.ComponentState()
        self.ComponentBalance()

        self.Wc= 0
        self.Qlow = 0

        for key in self.comps:
            if self.comps[key].energy == "CompressionWork":
                self.Wc += self.comps[key].Wc
            elif self.comps[key].energy == "RefrigerationCapacity":
                self.Qlow += self.comps[key].Qlow

        self.cop = self.Qlow / self.Wc
        self.Qlow += self.Qlow*60*(1/211)

    def __setformatstr(self, formatstr, result):
        result += formatstr.format('CompressionWork(kW)', self.Wc)
        result += formatstr.format('RefrigerationCapacity(ton)', self.Qlow)
        result += formatstr.format('cop', self.cop)
        return result

    def __str__(self):
        str_curtime = time.strftime(
            "%Y/%m/%d %H:%M:%S", time.localtime(time.time()))
        result = "\n Rankine Cycle: {}, Time: {}\n".format(
            self.name, str_curtime)
        try:
            formatstr = "{:>20} {:>.2f}\n"
            result = self.__setformatstr(formatstr, result)
        except:
            formatstr = "{} {}\n"
            result = self.__setformatstr(formatstr, result)
        return result


Overwriting ./refrigeration/vprcycle/cycleobj.py


In [16]:
%%file ./refrigeration/vprcycle/cyclehelper.py


"""
General Object-oriented Abstraction of vpr Cycle 

 - OutFiles(cycle, outfilename=None)
 - SpecifiedSimulator(cycle, prefixResultFileName, SetPower=None, SetMass=None):

Author:Cheng Maohua  Email: cmh@seu.edu.cn

"""
import sys
from components.node import Node

def OutFiles(cycle, outfilename=None):
    savedStdout = sys.stdout
    # redirect to the outfilename
    if outfilename is not None:
        datafile = open(outfilename, 'w', encoding='utf-8')
        sys.stdout = datafile

    # 1 output cycle performance
    print(cycle)
   
    # 2 output nodes
    print(Node.title)
    for key in cycle.nodes:
        print(cycle.nodes[key])
    # 3 output devices
    for key in cycle.comps:
        print(cycle.comps[key])
    
    # return to sys.stdout
    if (outfilename != None):
        datafile.close()
        sys.stdout = savedStdout


Overwriting ./refrigeration/vprcycle/cyclehelper.py


### 2.6 Cycle

In [12]:
%%file ./refrigeration/refrigerationapp.py
"""
General Object-oriented Abstraction  of VPR Cycle 

The Simulator of vpr Cycle 

  * Input :vpr cycle dict model

  * output: text file

 
Run: 

python refrigerationapp.py
  

"""
from vprcycle.cycleobj import RefrigerationCycle
from vprcycle.cyclehelper import OutFiles
from cyclemodel import *
from platform import *

if __name__ == "__main__":
       
    curpath = os.path.abspath(os.path.dirname(__file__))
    ResultFilePath =curpath+'\\'+'./result/'
      
    for curcycle in cycles:
        ResultFileName=ResultFilePath+curcycle.cycle['name']

        cycle = RefrigerationCycle(curcycle.cycle)
        cycle.simulator()
        # output to text
        OutFiles(cycle)
        OutFiles(cycle, ResultFileName + '.txt')



Overwriting ./refrigeration/refrigerationapp.py


## 3 The Projects

### The dir tree

```
   <refrigerationop>
      │ 
      |── rrefrigerationapp.py  #  main app
      |
      |── <components> components package
      │    |
      │    │ ── node.py
      │    |
      │    │ ── compressor.py
      │         .... 
      |── <cyclemodel> the vpr cycle dicts
      │     |
      │     │ ── vpr1.py # the cycle dict
      │           
      |── <vprcycle> cycle package
      │    |
      │    │ ─ cyclehelper.py # helper methods  
      │    │     
      │    │ ─ cycleobj.py # the object of cycle object
      │              
      |── <result>          
            |
            │ ─ vpr?.txt 
            │ 
```         
 
### The Flowchart  

```
   ┌─────────────────────────────────────┐ 
   │ Instance of Simulator using the dict│   
   │    * the instance of nodes          │
   │        The node state with tx │
   │    * the instance of devices        │ 
   └───────────────────┬─────────────────┘
                       ↓  
       ┌───────────────┴─────────────────┐ 
       │ The node state of device        │
       └───────────────┬─────────────────┘
                       ↓ 
   ┌───────────────────┴────────────────────┐ 
   │ The mass and energy balance of device  │ 
   │           on the mdot              │
   └──────────────────┬─────────────────────┘
                      ↓  
        ┌─────────────┴─────────────┐ 
        │ The performance of cycle  │ 
        │        on the mdot mass    │
        └─────────────┬─────────────┘
                      ↓  
       ┌──────────────┴───────────────┐ 
       │   Print results on console   │ 
       │   Save  results to text file│ 
       └──────────────────────────────┘
```
